In [ ]:
# Importing relevant libraries
%%time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
!pip install dateparser
import dateparser
!pip install usaddress
import usaddress
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import pickle
!pip install uszipcode
from uszipcode import SearchEngine
!pip install autocorrect
from autocorrect import Speller
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
!pip install catboost
from textblob import TextBlob
from textblob import Word
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
!pip install pyparsing

CPU times: user 123 ms, sys: 61.5 ms, total: 185 ms
Wall time: 20.2 s


In [ ]:
# Getting the dataset from csv or excel file
%%time
input_data = input("Enter the training data location:")
try:
   df = pd.read_csv(input_data) 
except:
   df = pd.read_excel(input_data)
# Getting the target variable
target_variable = input("Enter the target variable:")
# Dropping Nan values from target variable
df = df.dropna(subset = [target_variable])
# Creating another dataframe with just the target variable column 
df1 = df
# Dropping the target variable from original dataset
df = df.drop(target_variable, axis=1)
# Deciding whether target variable is categorical or continuous
cat_or_con = df1[target_variable].nunique()
if cat_or_con > 10: # Personal experience parameter
   cat_or_con = 'Continuous'
else:
   cat_or_con = 'Categorical'

Enter the training data location:/content/commentmess.xlsx
Enter the target variable:O
CPU times: user 165 ms, sys: 12.5 ms, total: 178 ms
Wall time: 8.89 s


In [ ]:
# Getting rid of duplicate rows and columns
%%time
df = df.drop_duplicates()
df = df.loc[:,~df.columns.duplicated()]

CPU times: user 3.54 ms, sys: 55 µs, total: 3.59 ms
Wall time: 5.65 ms


In [ ]:
# Removing white spaces
%%time
df = df.apply(lambda x: x.str.strip() if type(x) == str else x)

CPU times: user 1.98 ms, sys: 0 ns, total: 1.98 ms
Wall time: 1.98 ms


In [ ]:
# Removing columns with more than 60% missing values
%%time
df = df.dropna(thresh= 0.6 * len(df), axis=1)

CPU times: user 3.05 ms, sys: 4 µs, total: 3.05 ms
Wall time: 3 ms


In [ ]:
# Modified Date Engineering
%%time
def dates_engineering(df):
    # Selecting all columns with data or time in the name
    date_cols = [col for col in df.columns if ('date' in col.lower()) | ('time' in col.lower()) ]
    # Date columns are stored as objects. Hence, we need to convert them to datetime. Setting infer datetime format to true ensures
    # all different date formats are dealt with. Coercing ensures that any data impurity is converted to NaT.
    df[date_cols] = df[date_cols].applymap(lambda x:dateparser.parse(str(x)))
    # Removing NaT values(impurities) from data columns
    df[date_cols] = df[date_cols].dropna()
    # Removing columns with more than 60% missing values
    df = df.dropna(thresh= 0.6 * len(df), axis=1)
    # Selecting columns of datetime data type
    date_columns = list(df.select_dtypes(include=['datetime64[ns]']).columns)
    if len(date_columns) == 0: # Date engineering need not be done if there are no date columns
       pass
    elif len(date_columns) == 1: # Date engineering involves getting timedelta from date, which is done by subtracting from current date
       df[date_columns] = (df[date_columns] - np.datetime64('now'))
       df[date_columns] = df[date_columns].astype('timedelta64[D]')
    else: # If there are more than one date columns, they are reduced to single column by subtraction of dates and converted to timedelta
       df[date_columns].reduce(lambda x, y: x - y, l)
       df[date_columns] = (df[date_columns] - np.datetime64('now'))
       df[date_columns] = df[date_columns].astype('timedelta64[D]')
    return df
df = dates_engineering(df)

CPU times: user 672 ms, sys: 49.7 ms, total: 721 ms
Wall time: 723 ms


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Address  14 non-null     object
dtypes: object(1)
memory usage: 224.0+ bytes


In [ ]:
# Date Engineering
%%time
def dates_engineering(df):
    # Selecting all columns with data or time in the name
    date_cols = [col for col in df.columns if ('date' in col.lower()) | ('time' in col.lower()) ]
    # Date columns are stored as objects. Hence, we need to convert them to datetime. Setting infer datetime format to true ensures
    # all different date formats are dealt with. Coercing ensures that any data impurity is converted to NaT.
    df[date_cols] = df[date_cols].apply(lambda col: pd.to_datetime(col, infer_datetime_format=True, errors='coerce') 
              if col.dtypes == object 
              else col, 
              axis=0)
    # Removing NaT values(impurities) from data columns
    df[date_cols] = df[date_cols].dropna()
    # Removing columns with more than 60% missing values
    df = df.dropna(thresh= 0.6 * len(df), axis=1)
    # Selecting columns of datetime data type
    date_columns = list(df.select_dtypes(include=['datetime64[ns]']).columns)
    if len(date_columns) == 0: # Date engineering need not be done if there are no date columns
       pass
    elif len(date_columns) == 1: # Date engineering involves getting timedelta from date, which is done by subtracting from current date
       df[date_columns] = (df[date_columns] - np.datetime64('now'))
       df[date_columns] = df[date_columns].astype('timedelta64[D]')
    else: # If there are more than one date columns, they are reduced to single column by subtraction of dates and converted to timedelta
       df[date_columns].reduce(lambda x, y: x - y, l)
       df[date_columns] = (df[date_columns] - np.datetime64('now'))
       df[date_columns] = df[date_columns].astype('timedelta64[D]')
    return df
df = dates_engineering(df)

CPU times: user 20.7 ms, sys: 3.94 ms, total: 24.7 ms
Wall time: 37.7 ms


In [ ]:
df

,Comment
0,I like dogs
1,Dogs are luv
2,Cats are amazing
3,I hate cats
4,Rabbits are cute
5,Rabbits are cuter than dogs


In [ ]:
# Feature engineering address columns
#%%time
def address_engineering(df):
    # Selecting all columns with address in the name
    address_cols = [col for col in df.columns if 'address' in col.lower()]
    # Loading dictionary of US zipcodes with states
    search = SearchEngine()
    if len(address_cols) == 0: # Address engineering need not be done if there are no address columns
       pass
    elif len(address_cols) == 1: # Using regex to extract zipcode from address column and running it by the database to get state
         df['Zipcode'] = df[address_cols].applymap(lambda x: dict(usaddress.parse(str(x))))
         #df['Address_State'] = df['Zipcode'].apply(lambda x: zcdb[x].state
    else: # Concatenating address columns, using regex to extract zipcode from column and running it by the database to get state
         df['Zipcode'] = df[address_cols].apply(lambda x: ' '.join(x), axis = 1)
         df['Zipcode'] = df[address_cols].apply(lambda x: usaddress.parse(str(x)).ZipCode)
         df['Address_State'] = df['Zipcode'].apply(lambda x: zcdb[x].state)
    return df
df = address_engineering(df)  

In [ ]:
# Modified Feature engineering address columns
#%%time
def address_engineering(df):
    # Selecting all columns with address in the name
    address_cols = [col for col in df.columns if 'address' in col.lower()]
    # Loading dictionary of US zipcodes with states
    search = SearchEngine()
    if len(address_cols) == 0: # Address engineering need not be done if there are no address columns
         pass
    elif len(address_cols) == 1: # Using regex to extract zipcode from address column and running it by the database to get state
         df['Zipcode'] = df[address_cols].applymap(lambda x: re.findall('[0-9]{4,6}', str(x)))
         df['Zipcode'] = df['Zipcode'].apply(lambda x: x[-1] if len(x) != 0 else None)
         df['Zipcode'] = df['Zipcode'].apply(lambda col: pd.to_numeric(col, errors='coerce')).astype('Int64')
         df['Address_State'] = df['Zipcode'].apply(lambda x: search.by_zipcode(x).state)     
    else: # Concatenating address columns, using regex to extract zipcode from column and running it by the database to get state
         df['Zipcode'] = df[address_cols].apply(lambda x: ' '.join(x), axis = 1)
         df['Zipcode'] = df[address_cols].applymap(lambda x: re.findall('[0-9]{4,6}', str(x)))
         df['Zipcode'] = df['Zipcode'].apply(lambda x: x[-1] if len(x) != 0 else None)
         df['Zipcode'] = df['Zipcode'].apply(lambda col: pd.to_numeric(col, errors='coerce')).astype('Int64')
         df['Address_State'] = df['Zipcode'].apply(lambda x: search.by_zipcode(x).state)
    return df
df = address_engineering(df)     

In [ ]:
# Numeric Engineering
%%time
def numeric_engineering(df):
  # Getting categorical columns that should be numeric
  df = df.apply(lambda col: pd.to_numeric(col, errors='ignore') 
              if col.dtypes == object 
              else col, 
              axis=0)
  # Defining a dictionary mapping k, million etc. to numeric meaning
  num_map = {'k':1000, 'm':1000000, 'b':1000000000, 'thousand':1000, 'million':1000000, 'billion':1000000000}
  object_columns = list(df.select_dtypes(include=['object']).columns)
  for obj in object_columns: # Going thorugh all object columns
    if df[obj].str.contains('^[1-9][0-9]{1,}\s{0,}[k,m,b,million,billion,thousand]$', regex=True).any(axis=0) == True : 
       # If column has numbers followed by k ,million etc.
       df[obj] = df[obj].map(lambda x: (float(x[:-1]) * num_map.get(x[-1].lower(), 1))) # Replacing dictionary keys with values
       df[obj] = df[obj].astype('float64')
    # Getting numbers from gibberish
    if df[obj].str.contains('\d{1,}.?\d{1,}', regex=True).any(axis=0) == True:
       df[obj] = df[obj].str.extract(pat = '(\d{1,}\.?\d{1,})') 
       df[obj] = df[obj].astype('float64')
 # As far as numbers with commas go, I see no harm in applying the same over the whole dataframe
  try:
    df[object_columns].apply(lambda x: x.str.replace(',', '').astype(float), axis=1)
  except:
    pass
  return df
df = numeric_engineering(df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  6 non-null      object
dtypes: object(1)
memory usage: 96.0+ bytes


In [ ]:
# Dealing with comments
#%%time
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
def sentiment_classifier(x): # Ensuring continuous polarity is grouped into three classes of positive, neutral and negative
               if x > 0.1:
                  x = 1
               elif x < -0.1:
                  x = -1
               else:
                  x = 0
               return float(x)
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None
def NLP(df):
    comment_cols = [col for col in df.select_dtypes(include=['object']).columns if ('address' not in col.lower())]
    if len(comment_cols) > 0:
       for col in comment_cols: 
         #if df[col].nunique() > (0.5 * len(df)):
           #if df[col].str.contains('\d{1,}.?\d{1,}', regex=True).any(axis=0) == True:
              # Lowercasing and spellchecking
              df = df.applymap(lambda s:s.lower() if type(s) == str else s)
              spell = Speller()
              df = df.applymap(lambda s:spell(s) if type(s) == str else s)
              # Punctuation removal
              df[col] = df[col].str.replace('[^\w\s]','')
              # Stopword removal
              stop = stopwords.words('english')
              df[col] = df[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
              # Getting polarity
              Polarity = str(col) + 'Polarity'
              df[Polarity] = df[col].apply(sentiment_calc)
              df[Polarity] = df[Polarity].apply(lambda x: sentiment_classifier(x)) # Defining sentiment on basis of polarity
              # Lemmatization
              df[col] = df[col].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
              # Tokenization
              df[col] = df[col].apply(lambda x: TextBlob(x).words)
    return df, comment_cols
df, comment_cols = NLP(df)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df

,Comment,CommentPolarity
0,"[like, dog]",0.0
1,"[dog, lui]",0.0
2,"[cat, amazing]",1.0
3,"[hate, cat]",-1.0
4,"[rabbit, cute]",1.0
5,"[rabbit, cuter, dog]",0.0


In [ ]:
if len(comment_cols) > 0:
   for col in comment_cols: 
       #if df[col].nunique() > (0.5 * len(df)):
           # Creating document word matrix and saving the model
              cv = CountVectorizer(analyzer='word',       
                             min_df=3,
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}')
              df[col] = df[col].apply(lambda x: " ".join(x))
              cv.fit(df[col])
              cvname = 'cv_model.sav'
              pickle.dump(cv, open(cvname, 'wb'))
              X_LDA = cv.transform(df[col])
              # Applying LDA and saving the model
              from sklearn.decomposition import LatentDirichletAllocation
              lda_model = LatentDirichletAllocation(n_components=20,               
                                      learning_method='online',   
                                      random_state=100,          
                                      batch_size=128,                 
                                      n_jobs = -1)
              lda_model.fit(X_LDA)
              ldaname = 'lda_model.sav'
              pickle.dump(lda_model, open(ldaname, 'wb'))
              lda_output = lda_model.transform(X_LDA)
              topic_extraction = np.apply_along_axis(lda_output.index(max(lda_output)), 1, lda_output)
              # Getting topic
              df[col] = pd.DataFrame(topic_extraction)

ValueError: ignored

In [ ]:
# Removing object columns with more than 60 levels
%%time
ini_objects = list(df.select_dtypes(include=['object']))
for x in ini_objects:
    if df[x].nunique() > 60:
       df = df.drop(x, axis=1)

In [ ]:
# Identifying continuous and categorical variable columns
%%time
def ident(df):
 datatypes = [] # List where categorical or continuous info will be stored
 for var in df.columns:
    catorcon = df[var].nunique()
    if catorcon > 10: # Personal experience parameter
       catorcon = 1 # Continuous
    else:
       catorcon = 0 # Categorical
    datatypes.append(catorcon)
# Ensuring categorical columns are object dtype
 for i in range(len(datatypes)):
  if datatypes[i] == 0:
     df.iloc[:,i] = df.iloc[:,i].astype('category')
 return df,datatypes
df,datatypes = ident(df)

In [ ]:
# Lowercasing and spell checking all strings
%%time
df = df.applymap(lambda s:s.lower() if type(s) == str else s)
spell = Speller()
df = df.applymap(lambda s:spell(s) if type(s) == str else s)

In [ ]:
# Grouping categorical classes with less than 0.5% values into a singular class 'combined'
%%time
for i in datatypes:
  if i == 0:
     frequencies = df.iloc[:,i].value_counts(normalize=True)
     comfreq = frequencies[frequencies<0.005]
     df.iloc[:,i].replace(comfreq.index,'combined')

In [ ]:
# Removing columns with Pearson Correlation > 0.85
%%time
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr() # Correlation matrix
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr): # Elements above threshold
                colname = corr_matrix.columns[i] # Getting the name of column
                col_corr.add(colname) # To avoid removal of more columns than necessary, storing the column above threshold value 
                if colname in dataset.columns:
                    del dataset[colname] # Deleting the column from the dataset
    return dataset
df = correlation(df,0.85)
print(df)

In [ ]:
# Categorical imputation
%%time
objects = list(df.select_dtypes(include=['object']))
df[objects] = df[objects].fillna(value='missing') # Filling Nan values with missing

In [ ]:
# Splitting data into train and test
%%time
X = df
post_smote = X.columns
y = df1[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Performing 80/20 split

In [ ]:
# Power transform on numerical variables
%%time
fl_and_int = list(X_train.select_dtypes(include=['float','int'])) # Selecting numeric columns
a = 0
pt = PowerTransformer(method='yeo-johnson') # Using yeo johnson, since we can have both positive and negative values
if len(fl_and_int) > 0:
 pt.fit(X_train[fl_and_int]+.0001) # Adding this to ensure there are no zero values
 X_train[fl_and_int] = pt.transform(X_train[fl_and_int]+.0001)
 X_test[fl_and_int] = pt.transform(X_test[fl_and_int]+.0001) 
 a = 1
 ptname = 'pt_model.sav' # Saving the model
 pickle.dump(pt, open(ptname, 'wb'))
if cat_or_con == 'Continuous': # The target varibale needs to be transformed too if it is continuous
  pt1 = PowerTransformer(method='yeo-johnson') 
  pt1.fit(y_train.to_numpy().reshape(-1, 1) + .0001)
  y_train = pt1.transform(y_train.to_numpy().reshape(-1, 1) + .0001)
  y_test = pt1.transform(y_test.to_numpy().reshape(-1, 1) + .0001) 
  pt1name = 'pt1_model.sav' # Saving the model
  pickle.dump(pt1, open(pt1name, 'wb'))

In [ ]:
# Outlier removal
%%time
if len(fl_and_int) > 0: # Replacing values greater than 4 or lesser than -4 with Nan
 X_train[fl_and_int].where((np.abs(X_train[fl_and_int] > 4)) ,np.NaN, inplace=True)
 X_test[fl_and_int].where((np.abs(X_train[fl_and_int] > 4)) ,np.NaN, inplace=True)

In [ ]:
# Continuous Imputation, filling Nan values with median
%%time
X_train[fl_and_int] = X_train[fl_and_int].fillna(X_train[fl_and_int].median()) 
X_test[fl_and_int] = X_test[fl_and_int].fillna(X_test[fl_and_int].median()) 

In [ ]:
# Oversampling/Undersampling
%%time
if cat_or_con == 'Categorical':
   y_train = pd.Series(y_train)
   n = y_train.nunique() # Getting number of unique values
   class_percentage = y_train.value_counts(normalize=True) * 100 # Getting distribution of values
   target_level = class_percentage.min() # Finding class with least value count
   if target_level > (1/(50*n)):
      sm = RandomOverSampler(random_state=42)
      X_train, y_train = sm.fit_resample(X_train, y_train)
   else:
      sm = RandomOverSampler(random_state=42)
      X_train, y_train = sm.fit_resample(X_train, y_train)
      class_percentage = y_train.value_counts(normalize=True) * 100
      target_level = class_percentage.min()
      if target_level > (1/(50*n)):
         sm = RandomOverSampler(random_state=42)
         X_train, y_train = sm.fit_resample(X_train, y_train)
      else:
         um = RandomUnderSampler(random_state=42)
         X_train, y_train = um.fit_resample(X_train, y_train)
   X_train = pd.DataFrame(X_train, columns = post_smote)

In [ ]:
# After SMOTE, all columns become objects, which we don't want, so the numeric ones are converted back to numeric
X_train = X_train.apply(lambda col: pd.to_numeric(col, errors='ignore') 
              if col.dtypes == object 
              else col, axis=0)
y_train = pd.to_numeric(y_train.flatten(), errors='ignore')
y_train = pd.DataFrame(y_train, columns = [target_variable])
y_test = pd.DataFrame(y_test, columns = [target_variable])

In [ ]:
# Converting categorical columns to encoded ones via one hot encoding
%%time
categorical_cols = list(X_train.select_dtypes(include=['object']))
print(categorical_cols)
if len(categorical_cols) > 0:
   X_train = pd.concat([X_train.drop(categorical_cols, axis=1), pd.get_dummies(X_train[categorical_cols])], axis=1)
   X_test = pd.concat([X_test.drop(categorical_cols, axis=1), pd.get_dummies(X_test[categorical_cols])], axis=1)

In [ ]:
%%time
miss_cols = X_train.columns.difference(X_test.columns) #columns in X_test that are not in X_train
if len(miss_cols) > 0: # This only needs to be applied if there is actually a disparity between the train and test columns
   for col in miss_cols:
    try:
       m = int(X_train[col].mean()) # In case of missing columns in test, the same is generated with mean values of column in train
       X_test[col] = m
    except:
       m = int(X_test[col].mean()) # In case of missing columns in train, the same is generated with mean values of column in test
       X_train[col] = m
cols = X_train.columns.tolist()
X_test = X_test[cols] # Ensuring train and test have same columns

In [ ]:
# Feature selection
%%time
if cat_or_con == 'Categorical':
 model = DecisionTreeClassifier(max_depth = 50, min_samples_leaf = 5) 
else: 
 model = DecisionTreeRegressor(max_depth = 50, min_samples_leaf = 5)
model.fit(X_train, y_train)
feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
feat_importances.nlargest(5).plot(kind='barh') # Plotting top 5 important features on horizontal bar graph

In [ ]:
# Input for model selection, since model selection does not require us to try different models over whole dataset
%%time
if len(X_train) <= 10000:
  input_X_train = X_train
  input_y_train = y_train
elif len(X_train) > 10000 & len(X_train) <= 100000:
  input_X_train = X_train.sample(frac=0.8, random_state=1)
  input_y_train = y_train.sample(frac=0.8, random_state=1)
elif len(X_train) > 100000 & len(X_train) <= 1000000:
  input_X_train = X_train.sample(frac=0.7, random_state=1)
  input_y_train = y_train.sample(frac=0.7, random_state=1)
else:
  input_X_train = X_train.sample(frac=0.5, random_state=1)
  input_y_train = y_train.sample(frac=0.5, random_state=1)

In [ ]:
%%time
if cat_or_con == 'Continuous':
     import Regression
     name,mod,acc,par = Regression.best_model_reg(input_X_train.values, X_test.values, input_y_train.values.ravel(), y_test.values.ravel())
else:
     import classification
     name,mod,acc,par = classification.best_model_class(input_X_train.values, X_test.values, input_y_train.values.ravel(), y_test.values.ravel())

In [ ]:
acc

In [ ]:
# Fitting best model over whole dataset
mod.fit(X_train,y_train) 

In [ ]:
# Saving the model
filename = 'finalized_model.sav' 
pickle.dump(mod, open(filename, 'wb'))

In [ ]:
# Scoring test data
%%time
df1 = pd.read_csv(input("Enter the test data location:"))
y1 = df1[target_variable]
df1 = df1.loc[:,~df1.columns.duplicated()]
df1 = df1.apply(lambda x: x.str.strip() if type(x) == str else x)
df1 = dates_engineering(df1)
df1 = address_engineering(df1)
df1 = numeric_engineering(df1)
df1, comment_cols = NLP(df1)
if len(comment_cols) > 0:
   for col in comment_cols: 
       if df1[col].nunique() > (0.5 * len(df1)):
              cv = pickle.load(open(cvname, 'rb'))
              X_LDA = cv.transform(df1[col])
              lda = pickle.load(open(ldaname, 'rb'))
              lda_output = lda_model.transform(X_LDA)
              topic_extraction = np.apply_along_axis(lda_output.index(max(lda_output)), 1, lda_output)
              df1[col] = pd.DataFrame(topic_extraction)
ini_objects = list(df1.select_dtypes(include=['object']))
for x in ini_objects:
    if df1[x].nunique() > 60:
       df1 = df1.drop(x, axis=1)
df1,datatypes = ident(df1)
df1 = df1.applymap(lambda s:s.lower() if type(s) == str else s)
#spell = Speller()
#df1 = df1.applymap(lambda s:spell(s) if type(s) == str else s)
for i in range(len(datatypes)):
  if datatypes[i] == 0:
     df1.iloc[:,i] = df1.iloc[:,i].astype('category')
     frequencies = df1.iloc[:,i].value_counts(normalize=True)
     comfreq = frequencies[frequencies<0.005]
     df1.iloc[:,i].replace(comfreq.index,'combined')
df1[objects] = df1[objects].fillna(value='missing')
df1 = correlation(df1,0.9)
X1 = df1.drop(target_variable, axis=1)
if a==1:
 pt = pickle.load(open(ptname, 'rb'))
 X1[fl_and_int] = pt.transform(X1[fl_and_int])
if cat_or_con == 'Continuous':
 pt1 = pickle.load(open(pt1name, 'rb'))
 y1 = pt1.transform(y1.to_numpy().reshape(-1, 1))
X1[fl_and_int] = X1[fl_and_int].fillna(X1[fl_and_int].median()) 
if len(categorical_cols)>0:
 X1 = pd.concat([X1.drop(categorical_cols, axis=1), pd.get_dummies(X1[categorical_cols])], axis=1)
miss_cols = X_train.columns.difference(X1.columns) 
if len(miss_cols) > 0:
   for col in miss_cols:
    try:
       m = int(X_train[col].mean())
       X1[col] = m
    except:
       X1 = X1.drop(col, axis=1)
cols = X_train.columns.tolist()
X1 = X1[cols]

In [ ]:
# Making predictions on scoring dataset
loaded_model = pickle.load(open(filename, 'rb'))
abc = loaded_model.predict(X1)
abc = pt1.inverse_transform(abc)

In [ ]:
# Scoring
if cat_or_con == 'Continuous':
   from sklearn.metrics import r2_score
   X1.fillna(X1.median(),inplace=True)
   score = r2_score(y1, abc)
else:
   from sklearn.metrics import confusion_matrix
   X1.fillna(X1.median(),inplace=True)
   score = confusion_matrix(y1, abc)